# PhishHook - A Phishing URL Detector - UCI Dataset
#### By: Aryaan Khan and Bradley Lewis

## Import Libraries

In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

from phishhooknet import PhishHookNet

## Load the Data

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# Fetch the phishing website dataset from the UCI ML Repository
phiusiil_phishing_url_website = fetch_ucirepo(id=967) 
X = phiusiil_phishing_url_website.data.features 
y = phiusiil_phishing_url_website.data.targets

# Define which features to use
feature_names = [
    'URLLength', 'DomainLength', 'IsDomainIP',
    'TLDLength', 'NoOfSubDomain', 'HasObfuscation', 'IsHTTPS', 'NoOfEqualsInURL',
    'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
    'SpacialCharRatioInURL', 'NoOfLettersInURL', 'LetterRatioInURL',
    'NoOfDegitsInURL', 'DegitRatioInURL'
]
X = X[feature_names]


## Create the Dataloader

In [3]:
# Select the device to use
if torch.backends.mps.is_available():
    print("MPS is available!")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("CUDA is available!")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU instead.")
    device = torch.device("cpu")

# Split the data into temporary training data and final test data
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temporary training data into final training data and validation data
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Convert the columns to numeric values
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Handle missing values if there are any
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)
X_test = X_test.fillna(0)

# Convert data to tensors and transfer to the specified device
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Create Tensor datasets for all sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for all datasets
train_loader = DataLoader(train_dataset, batch_size=10000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=3000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=3000, shuffle=False)

# Confirm the DataLoader shapes
print(train_loader.dataset.tensors[0].shape, train_loader.dataset.tensors[1].shape)
print(val_loader.dataset.tensors[0].shape, val_loader.dataset.tensors[1].shape)
print(test_loader.dataset.tensors[0].shape, test_loader.dataset.tensors[1].shape)

MPS is available!
torch.Size([141477, 16]) torch.Size([141477, 1])
torch.Size([47159, 16]) torch.Size([47159, 1])
torch.Size([47159, 16]) torch.Size([47159, 1])


## Define the Training and Validation Loops

In [4]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(train_loader.dataset)

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            predicted = outputs.round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_loss = total_loss / len(val_loader.dataset)
    accuracy = correct / total * 100
    return avg_loss, accuracy

## Define the Early Stopping Condition Class

In [5]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''
        Saves model when validation loss decrease.
        '''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'phishing_uci_url_model.pth')
        self.val_loss_min = val_loss


## Train the Model

In [6]:
# Initialize the model
model = PhishHookNet(input_size=X_train.shape[1]).to(device)

# Loss function, optimizer, and early stopping
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.001)

In [7]:
# Training loop with model saving based on validation loss improvement
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, accuracy = validate(model, val_loader, criterion)
    
    # Early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    print(f'Epoch {epoch+1}: train_loss = {train_loss:.4f}, val_loss = {val_loss:.4f}')

Validation loss decreased (inf --> 0.351049).  Saving model ...
Epoch 1: train_loss = 0.8733, val_loss = 0.3510
Validation loss decreased (0.351049 --> 0.287188).  Saving model ...
Epoch 2: train_loss = 0.3719, val_loss = 0.2872
Validation loss decreased (0.287188 --> 0.082909).  Saving model ...
Epoch 3: train_loss = 0.2160, val_loss = 0.0829
Validation loss decreased (0.082909 --> 0.023130).  Saving model ...
Epoch 4: train_loss = 0.0574, val_loss = 0.0231
Validation loss decreased (0.023130 --> 0.019216).  Saving model ...
Epoch 5: train_loss = 0.0255, val_loss = 0.0192
EarlyStopping counter: 1 out of 5
Epoch 6: train_loss = 0.0202, val_loss = 0.0207
EarlyStopping counter: 2 out of 5
Epoch 7: train_loss = 0.0190, val_loss = 0.0197
Validation loss decreased (0.019216 --> 0.016870).  Saving model ...
Epoch 8: train_loss = 0.0184, val_loss = 0.0169
EarlyStopping counter: 1 out of 5
Epoch 9: train_loss = 0.0171, val_loss = 0.0176
EarlyStopping counter: 2 out of 5
Epoch 10: train_loss = 

## Evaluate the Model

In [8]:
# Load the best model back
model = PhishHookNet(input_size=X_train.shape[1]).to(device)
model.load_state_dict(torch.load('phishing_uci_url_model.pth'))

# Evaluate the model on the test dataset
model.eval()

# Calculate accuracy
train_accuracy = validate(model, train_loader, criterion)[1]
val_accuracy = validate(model, val_loader, criterion)[1]
test_accuracy = validate(model, test_loader, criterion)[1]

# Print the results
print(f'Training accuracy: {train_accuracy:.2f}%')
print(f'Validation accuracy: {val_accuracy:.2f}%')
print(f'Test accuracy: {test_accuracy:.2f}%')

Training accuracy: 99.74%
Validation accuracy: 99.69%
Test accuracy: 99.74%
